#### [사전 학습 - AlexNet Model 기반 Cat & Dog 분류]

[1] 모듈 로딩 및 데이터 준비 <hr>

In [3]:
## 모듈 로딩 및 데이터 준비
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchvision.transforms as transforms					## 이미지 전처리 변형 관련
from torchvision.models import alexnet, AlexNet_Weights		## 사전학습 내장 모델 클래스 관련
from torchinfo import summary								## 모델 정보 확인용

from torchvision.datasets import ImageFolder				## 커스텀 이키지 데이터셋 관련 모듈
from torch.utils.data import DataLoader

In [4]:
## 데이터 준비
DATA_DIR = '../data/image/animals/'

## 이미지 전처리 준비
PREPROCESSING = AlexNet_Weights.DEFAULT.transforms()

## 이미지 클래스 변환
IDX_TO_CLASS = dict(zip( range(1000), AlexNet_Weights.DEFAULT.meta['categories']))

[2] 이미지 데이터 로딩 <hr>

In [5]:
## - 이미지 데이터 로딩
imgDS = ImageFolder(root=DATA_DIR,
					transform=PREPROCESSING)

## - 클래스 변환 데이터
IDX_TO_CLASS = {v:k for k, v in imgDS.class_to_idx.items()}
print(f'IDX_TO_CLASS => {IDX_TO_CLASS}')

IDX_TO_CLASS => {0: 'cat', 1: 'dog'}


In [8]:
## - 데이터 확인
print(f'imgDataset 개수 : {len(imgDS.targets)}개')
print(f'imgDataset 분류 : {imgDS.class_to_idx}')
print(f'- cat 	   개수 : {imgDS.targets.count(0)}개, {(imgDS.targets.count(0)/len(imgDS.targets))*100}')
print(f'- dog	   개수 : {imgDS.targets.count(1)}개, {(imgDS.targets.count(1)/len(imgDS.targets))*100}')

imgDataset 개수 : 124개
imgDataset 분류 : {'cat': 0, 'dog': 1}
- cat 	   개수 : 62개, 50.0
- dog	   개수 : 62개, 50.0


In [13]:
## 데이터 확인
img, label = imgDS[0]
print(f'img => {img.shape}, label => {label}')

img => torch.Size([3, 224, 224]), label => 0


[3] 모댈 클래스 정의 및 선언 <hr>

In [ ]:
## 모델 구성 : 사전 학습된 AlexNet의 특징추출 부분 + 커스텀 분류기 구분
##			 - 특징추출 부분 : 최적의 W,b 설정 => 학습 X
##			 - 분류기   부분 : 커스터마이징	   => 학습 필요

In [14]:
model = alexnet(weights=AlexNet_Weights.DEFAULT)

In [21]:
## 모델 층별 W, b 파라미터 업데이트 설정
## ==> model.features 부분 : requires_grad = False
## ==> model.classifier 부분 : 2진 분류기로 변경
for name, param in model.named_parameters():
	if name.startswith('features'):
		param.requires_grad = False

## 확인
for name, param in model.named_parameters():
	print(name, param.shape, param.requires_grad)

features.0.weight torch.Size([64, 3, 11, 11]) False
features.0.bias torch.Size([64]) False
features.3.weight torch.Size([192, 64, 5, 5]) False
features.3.bias torch.Size([192]) False
features.6.weight torch.Size([384, 192, 3, 3]) False
features.6.bias torch.Size([384]) False
features.8.weight torch.Size([256, 384, 3, 3]) False
features.8.bias torch.Size([256]) False
features.10.weight torch.Size([256, 256, 3, 3]) False
features.10.bias torch.Size([256]) False
classifier.weight torch.Size([2, 9216]) True
classifier.bias torch.Size([2]) True


In [ ]:
## (2) 분류기 부분 변경 => 입력 (85, 9216) =====> 출력 ( ? , 2)
model.classifier = nn.Linear(9216, 2)

In [18]:
summary(model, input_size=(1, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
AlexNet                                  [1, 2]                    --
├─Sequential: 1-1                        [1, 256, 6, 6]            --
│    └─Conv2d: 2-1                       [1, 64, 55, 55]           23,296
│    └─ReLU: 2-2                         [1, 64, 55, 55]           --
│    └─MaxPool2d: 2-3                    [1, 64, 27, 27]           --
│    └─Conv2d: 2-4                       [1, 192, 27, 27]          307,392
│    └─ReLU: 2-5                         [1, 192, 27, 27]          --
│    └─MaxPool2d: 2-6                    [1, 192, 13, 13]          --
│    └─Conv2d: 2-7                       [1, 384, 13, 13]          663,936
│    └─ReLU: 2-8                         [1, 384, 13, 13]          --
│    └─Conv2d: 2-9                       [1, 256, 13, 13]          884,992
│    └─ReLU: 2-10                        [1, 256, 13, 13]          --
│    └─Conv2d: 2-11                      [1, 256, 13, 13]         

[4] 학습 준비 <hr>

In [22]:
## 학습 관련 설정 값
EPOCHS = 10
BATCH_SIZE = 32
LEARNING_RATE = 0.001
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [23]:
## 학습 관련 인스턴스들 생성
t_loader = DataLoader(imgDS, batch_size=BATCH_SIZE, shuffle=True)

## 손실 함ㅅ와 옵티마이저
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

## 모델 장치 할당
model = model.to(DEVICE)

In [24]:
## 학습 관련 함수들
def train_one_epoch(model, loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    acc = 100 * correct / total
    return total_loss, acc

[5] 학습 <hr>

In [26]:
for epoch in range(EPOCHS):
    loss, acc = train_one_epoch(model, t_loader, criterion, optimizer, DEVICE)
    print(f"[{epoch+1}/{EPOCHS}] Loss: {loss:.4f}, Accuracy: {acc:.2f}%")

[1/10] Loss: 8.9251, Accuracy: 59.68%
[2/10] Loss: 1.7563, Accuracy: 86.29%
[3/10] Loss: 0.0170, Accuracy: 100.00%
[4/10] Loss: 0.2751, Accuracy: 98.39%
[5/10] Loss: 0.0194, Accuracy: 100.00%
[6/10] Loss: 0.0009, Accuracy: 100.00%
[7/10] Loss: 0.0002, Accuracy: 100.00%
[8/10] Loss: 0.0001, Accuracy: 100.00%
[9/10] Loss: 0.0000, Accuracy: 100.00%
[10/10] Loss: 0.0000, Accuracy: 100.00%
